# Extract a PDF Document

**The old way:** Open the PDF, manually copy text, struggle to search tables, screenshot drawings one page at a time.

**The new way:** Upload a PDF to Istari, run extraction, get structured text, sections, semantic chunks, and metadata automatically.

This notebook:
1. Navigates the Istari system hierarchy to find the PDF file
2. Runs PDF data extraction using the Istari open_pdf tool
3. Views the 10 extracted artifacts (text, sections, chunks, OCR, HTML, metadata)
4. Snapshots the results so the team can see what was extracted

See [`example-output/`](example-output/) for pre-computed results.

In [ ]:
# Setup
import sys, json
from pathlib import Path

try:
    import istari_digital_client
except ImportError:
    !pip install istari-digital-client python-dotenv -q

repo_root = str(Path.cwd().parent.parent)
if repo_root not in sys.path:
    sys.path.insert(0, repo_root)

from istari_client import get_client

client = get_client()
user = client.get_current_user()
print(f"Connected as: {user.display_name} ({user.email})")

In [ ]:
# Explore the system — navigate the Istari hierarchy
#
# This system tracks one PDF document (BRACKET_v2.2.pdf).
#
# Istari links:
#   System: https://demo.istari.app/systems/94a23774-633b-46ed-9c55-1431649ac898

SYSTEM_ID = "94a23774-633b-46ed-9c55-1431649ac898"  # Example: Extract PDF
CONFIG_ID = "15dd6217-84b6-45cf-b61d-c4e11213142b"  # Baseline configuration

system = client.get_system(SYSTEM_ID)
print(f"System: {system.name}")
print(f"  {system.description}\n")

configs = client.list_system_configurations(SYSTEM_ID, page=1, size=50)
print(f"Configurations ({configs.total}):\n")

for config in configs.items:
    print(f"  {config.name}")
    print(f"    Config ID: {config.id}")

    tracked = client.list_tracked_files(config.id, page=1, size=50)
    print(f"    Tracked files ({tracked.total}):")
    for tf in tracked.items:
        mode = tf.specifier_type.value
        print(f"      {tf.file_id} ({mode})")

    snapshots = client.list_snapshots(configuration_id=config.id, page=1, size=10)
    print(f"    Snapshots ({snapshots.total}):")
    for snap in snapshots.items:
        tags = client.list_tags(snapshot_id=snap.id, page=1, size=10)
        tag_names = [t.tag for t in tags.items]
        tag_str = f"  [{', '.join(tag_names)}]" if tag_names else ""
        revs = client.list_snapshot_revisions(snap.id, page=1, size=50)
        print(f"      {snap.id[:8]}...{tag_str}  ({revs.total} file(s))")
        for r in revs.items:
            size_kb = r.size / 1024
            print(f"        - {r.name} ({size_kb:.1f} KB)")

In [ ]:
# Load the model — show file details and revision history

MODEL_ID = "f6804715-ec11-4ba2-847b-93be0f72e03f"  # BRACKET_v2.2.pdf

model = client.get_model(MODEL_ID)
name = model.display_name or model.file.revisions[0].name
print(f"Model: {name}")
print(f"  File ID:   {model.file.id}")
print(f"  Revisions: {len(model.file.revisions)}")
for rev in model.file.revisions:
    size_kb = rev.size / 1024
    ver = f"  ({rev.version_name})" if hasattr(rev, 'version_name') and rev.version_name else ""
    print(f"    {rev.name} — {size_kb:.1f} KB{ver}")
print(f"  Artifacts: {len(model.artifacts)}")
print(f"\nTracked in config: {CONFIG_ID[:8]}... (LATEST)")

In [ ]:
# Run PDF extraction
#
# Sends the PDF to an Istari agent running the PDF extractor.
# Extracts: structured text, sections, semantic chunks, metadata, OCR text, and HTML.
#
# Docs: https://docs.istaridigital.com/integrations/documents/open_pdf

from time import sleep
from datetime import datetime
from istari_digital_client import JobStatusName

print("Submitting PDF extraction job...")
job = client.add_job(
    model_id=MODEL_ID,
    function="@istari:extract",
    tool_name="open_pdf",
    tool_version="1.0.0",
    operating_system="Ubuntu 22.04",
    parameters={},
)
print(f"Job: {job.id}")

while True:
    sleep(10)
    job = client.get_job(job.id)
    ts = datetime.now().strftime("%H:%M:%S")
    status = job.status.name.value
    print(f"  [{ts}] {status}")
    if job.status.name in {JobStatusName.COMPLETED, JobStatusName.FAILED}:
        break

if job.status.name == JobStatusName.COMPLETED:
    print("\nExtraction complete!")
else:
    print("\nExtraction failed!")

In [ ]:
# View extraction results
#
# The agent produced 10 artifacts:
#   document.html, document.pdf, docling_text.json, text.txt, metadata.json,
#   text_sections.json, json_sections.json, smart_chunks.json,
#   semantic_chunks.json, text_with_OCR.json

model = client.get_model(MODEL_ID)
print(f"Artifacts: {len(model.artifacts)}\n")

for a in model.artifacts:
    rev = a.file.revisions[0] if a.file.revisions else None
    if rev:
        size_kb = rev.size / 1024
        ext = rev.name.split('.')[-1].lower() if '.' in rev.name else ''
        print(f"  {rev.name} ({size_kb:.1f} KB)")

        # Show JSON contents inline (500 char limit)
        if ext == 'json':
            try:
                data = json.loads(a.read_text())
                dumped = json.dumps(data, indent=2)
                print(f"    {dumped[:500]}")
                if len(dumped) > 500:
                    print("    ...")
            except Exception:
                pass
            print()

In [ ]:
# Snapshot: capture state after extraction

from istari_digital_client import NewSnapshot, NewSnapshotTag

print("Creating snapshot: post-extraction...")
snap_response = client.create_snapshot(CONFIG_ID, NewSnapshot())
snapshot = snap_response.actual_instance

if hasattr(snapshot, "id"):
    snap_id = snapshot.id
    print(f"  New snapshot: {snap_id[:8]}...")
else:
    snaps = client.list_snapshots(configuration_id=CONFIG_ID, page=1, size=1)
    snap_id = snaps.items[0].id
    print(f"  No changes — tagging existing snapshot: {snap_id[:8]}...")

client.create_tag(snap_id, NewSnapshotTag(tag="post-extraction"))
print("  Tagged: post-extraction")

revs = client.list_snapshot_revisions(snap_id, page=1, size=50)
print(f"  Files: {revs.total}")

In [ ]:
# View system state — all snapshots with tags, files, and sizes

print(f"System: {system.name}")
print(f"  {system.description}\n")

configs = client.list_system_configurations(SYSTEM_ID, page=1, size=50)
for config in configs.items:
    print(f"Configuration: {config.name}")
    print(f"  ID: {config.id}")

    tracked = client.list_tracked_files(config.id, page=1, size=50)
    print(f"  Tracked files: {tracked.total}")

    snapshots = client.list_snapshots(configuration_id=config.id, page=1, size=20)
    print(f"\n  Snapshots ({snapshots.total}):\n")

    for i, snap in enumerate(snapshots.items):
        tags = client.list_tags(snapshot_id=snap.id, page=1, size=10)
        tag_names = [t.tag for t in tags.items]
        tag_str = ", ".join(tag_names) if tag_names else "untagged"
        revs = client.list_snapshot_revisions(snap.id, page=1, size=50)

        print(f"    {i+1}. [{tag_str}]  ({revs.total} files)")
        for r in revs.items:
            size_kb = r.size / 1024
            if size_kb > 1024:
                print(f"       {r.name} ({size_kb/1024:.1f} MB)")
            else:
                print(f"       {r.name} ({size_kb:.1f} KB)")
        print()

## Summary

**What we did:**
1. Uploaded `BRACKET_v2.2.pdf` to Istari
2. Ran automated extraction — structured text, sections, semantic chunks, metadata, OCR text, and HTML
3. Viewed all 10 artifacts without manually reading the PDF
4. Snapshotted the extraction results for the team

### Version History

| # | Snapshot Tag | Files | What happened |
|---|-------------|-------|---------------|
| 1 | `initial-upload` | 1 | Raw BRACKET_v2.2.pdf uploaded |
| 2 | `post-extraction` | 11 | Original PDF + 10 extracted artifacts |

### What Istari Did

| Step | Inner Loop (open_pdf on Ubuntu 22.04) | Outer Loop (Istari) |
|------|--------------------------------------|---------------------|
| Upload document | — | Stored BRACKET_v2.2.pdf, tracked as LATEST |
| Extract data | Parsed PDF pages, ran OCR, segmented sections, chunked text, rendered HTML | Ran job on Linux agent, stored all 10 artifacts |
| Snapshot | — | Captured point-in-time state with all 11 files |

### What's Next?

- **Search content** — use `text.txt` or `docling_text.json` to programmatically search document content
- **Semantic search** — use `semantic_chunks.json` to build a vector index for similarity search
- **Compare revisions** — upload an updated PDF, re-extract, and compare sections across versions